# Analyse de game professionelles de League of Legends

In [1]:
import pandas as pd 

dataset = pd.read_csv("data/lol_pre_match_dataset.csv").dropna()
print(f"Il y a {len(dataset)} parties prêtes à l'analyse.")
mapped_dataset = dataset.copy()

Il y a 8971 parties prêtes à l'analyse.


In [5]:
print(f"Pour chaque ligne, voici les informaions que l'on a :\n{[col for col in dataset.columns]}")
# 0 = Blue win, 1 = Red win

Pour chaque ligne, voici les informaions que l'on a :
['gameid', 'team_blue', 'team_red', 'pick_blue_1', 'pick_blue_2', 'pick_blue_3', 'pick_blue_4', 'pick_blue_5', 'pick_red_1', 'pick_red_2', 'pick_red_3', 'pick_red_4', 'pick_red_5', 'ban_blue_1', 'ban_blue_2', 'ban_blue_3', 'ban_blue_4', 'ban_blue_5', 'ban_red_1', 'ban_red_2', 'ban_red_3', 'ban_red_4', 'ban_red_5', 'result']


In [3]:
# Liste de tous les champions_set qui apparaissent dans le dataset
# Liste de tous les champions qui apparaissent dans le dataset
champions_set = set()
for col in [
    "pick_blue_1", "pick_blue_2", "pick_blue_3", "pick_blue_4", "pick_blue_5",
    "pick_red_1", "pick_red_2", "pick_red_3", "pick_red_4", "pick_red_5",
    "ban_blue_1", "ban_blue_2", "ban_blue_3", "ban_blue_4", "ban_blue_5",
    "ban_red_1", "ban_red_2", "ban_red_3", "ban_red_4", "ban_red_5"
]:
    for champ in dataset[col]:
        champ = str(champ).strip()
        if champ:
            champions_set.add(champ)

print(f"Nous allons mapper aléatoirement le nom des {len(champions_set)} champions utilisés dans le dataset.")
champions_dict = {champ: id for id, champ in enumerate(champions_set)}
print(f"Voici un exemple de mapping : {list(champions_dict.items())[:5]}")

# Remplacement des noms par les IDs dans chaque colonne pick/ban
for col in [
    "pick_blue_1", "pick_blue_2", "pick_blue_3", "pick_blue_4", "pick_blue_5",
    "pick_red_1", "pick_red_2", "pick_red_3", "pick_red_4", "pick_red_5",
    "ban_blue_1", "ban_blue_2", "ban_blue_3", "ban_blue_4", "ban_blue_5",
    "ban_red_1", "ban_red_2", "ban_red_3", "ban_red_4", "ban_red_5"
]:
    mapped_dataset[col] = dataset[col].apply(lambda x: champions_dict.get(str(x).strip(), -1))

print(f"\nVoici un exemple de ligne du dataset mappé :\n{mapped_dataset.iloc[0]}")


Nous allons mapper aléatoirement le nom des 169 champions utilisés dans le dataset.
Voici un exemple de mapping : [('Graves', 0), ('Hwei', 1), ('Nocturne', 2), ('Senna', 3), ('Alistar', 4)]

Voici un exemple de ligne du dataset mappé :
gameid         10660-10660_game_1
team_blue             LNG Esports
team_red                Rare Atom
pick_blue_1                   145
pick_blue_2                    63
pick_blue_3                   104
pick_blue_4                   121
pick_blue_5                     3
pick_red_1                     30
pick_red_2                    129
pick_red_3                    107
pick_red_4                     97
pick_red_5                     39
ban_blue_1                     67
ban_blue_2                      2
ban_blue_3                    140
ban_blue_4                     47
ban_blue_5                     28
ban_red_1                       6
ban_red_2                      82
ban_red_3                     167
ban_red_4                     162
ban_red_5       

In [4]:
teams_set = set()
for col in ["team_blue", "team_red"]:
    for cell in dataset[col]:
        if isinstance(cell, str):
            teams_set.add(cell.strip())
print(f"Idem pour les {len(teams_set)} noms d'équipe.")
teams_dict = {}
for id, team in enumerate(teams_set):
    teams_dict[team] = id
print(f"Voici un exemple de mapping : {list(teams_dict.items())[:3]}")
def change_teams_ids(teams):
    return teams_dict.get(teams, -1)
for col in ["team_blue", "team_red"]:
    mapped_dataset[col] = dataset[col].apply(change_teams_ids)
print("")
print(f"Voici un exemple de ligne du dataset mappé :\n{mapped_dataset.iloc[0]}")

Idem pour les 469 noms d'équipe.
Voici un exemple de mapping : [('V3 Esports', 0), ('OKSavingsBank BRION Academy', 1), ('Team DX', 2)]

Voici un exemple de ligne du dataset mappé :
gameid         10660-10660_game_1
team_blue                      28
team_red                      377
pick_blue_1                   145
pick_blue_2                    63
pick_blue_3                   104
pick_blue_4                   121
pick_blue_5                     3
pick_red_1                     30
pick_red_2                    129
pick_red_3                    107
pick_red_4                     97
pick_red_5                     39
ban_blue_1                     67
ban_blue_2                      2
ban_blue_3                    140
ban_blue_4                     47
ban_blue_5                     28
ban_red_1                       6
ban_red_2                      82
ban_red_3                     167
ban_red_4                     162
ban_red_5                     109
result                          1
Nam